In [ ]:
!pip install mediapipe opencv-python-headless==4.7.0.72 numpy fpdf


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.8 MB/s eta 0:00:00
  Created wheel for fpdf: file

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from fpdf import FPDF
from google.colab import files

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a); b = np.array(b); c = np.array(c)
    ba = a - b; bc = c - b
    denom = (np.linalg.norm(ba) * np.linalg.norm(bc))
    if denom == 0:
        return 0.0
    cosine_angle = np.dot(ba, bc) / denom
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.degrees(np.arccos(cosine_angle))
    return angle


/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(


In [ ]:
def detect_pose_and_draw(video_path, save_frames=False):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    frames=[]
    angles=[]
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            # Using right side landmarks (11 shoulder,13 elbow,15 wrist are left in Mediapipe ordering depends on side)
            # We'll compute both left and right elbow angles (landmark indices differ by side)
            # Left side: shoulder=11, elbow=13, wrist=15 ; Right side: shoulder=12, elbow=14, wrist=16
            left_shoulder = [lm[11].x, lm[11].y]
            left_elbow = [lm[13].x, lm[13].y]
            left_wrist = [lm[15].x, lm[15].y]
            right_shoulder = [lm[12].x, lm[12].y]
            right_elbow = [lm[14].x, lm[14].y]
            right_wrist = [lm[16].x, lm[16].y]

            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # choose the side with higher confidence by comparing visibility if available
            angles.append({'left': left_elbow_angle, 'right': right_elbow_angle})

            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            cv2.putText(frame, f"L_elbow:{int(left_elbow_angle)} R_elbow:{int(right_elbow_angle)}",
                        (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        else:
            angles.append({'left': None, 'right': None})

        frames.append(frame)

    cap.release()
    return frames, angles


In [ ]:
def evaluate_elbow(angle):
    if angle is None:
        return "No detection"
    if angle < 45:
        return "Too Low - incomplete"
    elif 45 <= angle <= 160:
        return "Good Rep"
    else:
        return "Too Extended"

def overlay_and_save(frames, angles, out_path="/content/output_video.mp4", fps=25):
    h, w = frames[0].shape[:2]
    writer = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w,h))
    for frame, a in zip(frames, angles):
        # pick left if present else right
        angle = a.get('left') if (a.get('left') not in (None, 0.0)) else a.get('right')
        feedback = evaluate_elbow(angle)
        cv2.putText(frame, feedback, (10, h-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        writer.write(frame)
    writer.release()
    print("Saved:", out_path)
    return out_path


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sample_video.mp4 to sample_video.mp4


In [ ]:
frames, angles = detect_pose_and_draw("sample_video.mp4")
overlay_and_save(frames, angles)

Saved: /content/output_video.mp4


'/content/output_video.mp4'

In [ ]:
files.download("output_video.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>